## Test per progetto AI

E' necessario caricare i file roads_small.json e nodes_small.json

In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [4]:
nodes = json.load(open('new_nodes.json', 'r'))
roads = json.load(open('new_roads.json', 'r'))
print("Num nodes", len(nodes))
print("Num roads", len(roads))

Num nodes 2239
Num roads 10138


Create two maps as lookup tables for the nodes

In [5]:
#create a lookup table for point coordinates
node_coords = {point['id']:tuple(point['coordinates']) for point in nodes} #dictonary initialization with comprehension
coords_node = {tuple(point['coordinates']):point['id'] for point in nodes} #tuples are hashable-> can be key to a dictionary

In [6]:
print(node_coords[roads[11]['p1']]) #retrieve coordinates from node ID

(12.55, 45.59)


A questo punto bisogna generare le query degli utenti:

semplificazione: l'orario non influisce sui tempi di percorrenza -> dipendono solo dalla distanza

Consider each trip as starting or ending in one of the nodes: the function below serves to approximate the closest node on the map

Regular transfer (not related to pendolarism) is modeled as a gaussian random variable with respect to location ($\mu = 15km, \sigma = 10km$)  

In [7]:
# compute distance between two nodes

def compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2):
  x_coord_1 = x_coord_1 * 111.32
  x_coord_2 = x_coord_2 * 111.32
  y_coord_1 = 40075 * math.cos(x_coord_1)/360 * y_coord_1;
  y_coord_2 = 40075 * math.cos(x_coord_2)/360 * y_coord_2;
  dist = math.sqrt((y_coord_1 - y_coord_2)**2 + (x_coord_1 - x_coord_2)**2)
  return dist

In [8]:
# use of networkx, creation of graph

import networkx as nx
# use of networkx

G = nx.Graph()

G.add_nodes_from(range(len(nodes)))
for road in roads :
  n1 = int(road.get("p1"))
  n2 = int(road.get("p2"))
  x_coord_1 = float(nodes[n1].get('coordinates')[0])
  y_coord_1 = float(nodes[n1].get('coordinates')[1])
  x_coord_2 = float(nodes[n2].get('coordinates')[0])
  y_coord_2 = float(nodes[n2].get('coordinates')[1])
  dist = compute_distance(x_coord_1, y_coord_1, x_coord_2, y_coord_2)

  G.add_edge(n1, n2, weight=dist)

G.get_edge_data(0, 1)



{'weight': 2698.256500971123}

In [9]:
def generate_ratings(n_driver):
  mean, std = 3.5, 0.5
  x = np.random.normal(mean, std, size=(n_driver, 1))
  ratings = np.empty(shape = (n_driver, 1))
  for i in range(n_driver):
    y = math.ceil(x[i])
    if (y - x[i] > 0.5) :
      ratings[i] = y - 0.5
    else :
      ratings[i] = y
      
  return ratings


In [85]:
# pool of end points 
# define a random number of destinations (e.g. from 6 to 8)
num_dest = random.randrange(6, 8, 1)
pool_dest = []
for i in range(num_dest):
  rand = random.randrange(len(node_coords))
  id_dest = list(node_coords.keys())[rand]
  coord_dest = list(node_coords.values())[rand]
  pool_dest.append({'id':id_dest, 'coordinates':coord_dest})

print(pool_dest)

[{'id': 585, 'coordinates': (11.86, 45.56)}, {'id': 270, 'coordinates': (11.8, 45.35)}, {'id': 1519, 'coordinates': (12.05, 45.42)}, {'id': 479, 'coordinates': (12.39, 45.53)}, {'id': 1930, 'coordinates': (12.48, 45.64)}, {'id': 2186, 'coordinates': (12.21, 45.53)}, {'id': 137, 'coordinates': (12.03, 45.29)}]


Minimum path lenght:
- Drivers: 40 km
- Riders: 10 km

In [86]:
from datetime import datetime
from datetime import timedelta

In [87]:
def generate_trips(n, gen):
  # needs to define, for each driver/rider, one random starting point, and peak from the pool of destinations a random end points
  trips_start = []
  trips_dest = []
  for i in range(n):
    # start point
    if (gen == 'driver') :
      while True:
        # start point
        rand_s = random.randrange((len(nodes)))
        id_start = list(node_coords.keys())[rand_s]
        # end point
        rand_d = random.randrange(num_dest)
        dest = pool_dest[rand_d]
        dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
        if (dist > 40e3) :
          coord_start = list(node_coords.values())[rand_s]
          break
    else :
        while True:
          # start point
          rand_s = random.randrange((len(nodes)))
          id_start = list(node_coords.keys())[rand_s]
          # end point
          rand_d = random.randrange(num_dest)
          dest = pool_dest[rand_d]
          dist = nx.shortest_path_length(G, id_start, dest.get('id'), weight='weight')
          if (dist > 10e3) :
            coord_start = list(node_coords.values())[rand_s]
            break
     
    
    trips_start.append({'id':id_start, 'coordinates':coord_start})
    trips_dest.append(dest)

  return trips_start, trips_dest


In [88]:
def generate_schedules(n):
  
  #time probability mass: how many requests per 2-hour slot in percentage
  time_prob = {   
                  '00:00': 1e-3,
                  '2:00': 1e-3,
                  '4:00': 1e-3,
                  '6:00': 1e-3,
                  '6:30': 5e-3,
                  '7:00': 0.02,
                  '7:30': 0.1,
                  '8:00': 0.1,
                  '8:30': 0.1,
                  '10:00': 0.01,
                  '10:30': 0.01,
                  '12:00': 0.1,
                  '12:30': 0.1,
                  '13:00': 0.1,
                  '14:00': 0.01,
                  '16:00': 0.01,
                  '18:00': 0.1,
                  '18:30': 0.1,
                  '19:00': 0.1,
                  '19:30': 0.02,
                  '20:00': 5e-3,
                  '21:00': 5e-3,
                  '22:00': 1e-3,
  }

   


  times = []
  hours = list(time_prob.keys())
  prob = list(time_prob.values())
  for i in range(n):
    times.append((random.choices(hours, prob)[0]))

  return times

In [89]:
class Driver:
  def __init__(self, id, rating, hour, start_point, end_point):
    self.id = id
    self.rating = rating
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    return "Driver ( id = " + str(self.id) + ", rating = " + str(self.rating) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

class Rider:
  def __init__(self, id, hour, start_point, end_point):
    self.id = id
    self.hour = hour
    self.start_point = start_point
    self.end_point = end_point

  def __str__(self):
    date_format_str = '%H:%M'
    return "Rider ( id = " + str(self.id) + ", hour = " + self.hour + ", start = " + str(self.start_point) + ", dest = " + str(self.end_point) + ")"

In [90]:
def generate_data():
  # 1- generate n_drivers and n_riders
  # range: start, end, stepsize
  n_drivers = random.randrange(40, 50, 1)
  n_riders = random.randrange(90, 150, 15)
  print("N_drivers: ", n_drivers)
  print("N_riders: ", n_riders)
  # 2 - generate start point and end point for drivers and riders
  trips_start_drivers, trips_dest_drivers = generate_trips(n_drivers, gen='driver')
  trips_start_riders, trips_dest_riders = generate_trips(n_riders, gen='rider')
  # 3 - generate schedules
  drivers_schedule = generate_schedules(n_drivers)
  riders_schedule = generate_schedules(n_riders)
  # 4- generate ratings for drivers
  ratings = generate_ratings(n_drivers)
  # 5 - parse in a list of objects
  riders = []
  drivers = []
  for i in range(n_riders):
    riders.append( Rider(i, riders_schedule[i], trips_start_riders[i], trips_dest_riders[i]))

  for i in range(n_drivers):
    drivers.append( Driver(i, ratings[i][0], drivers_schedule[i], trips_start_drivers[i], trips_dest_drivers[i]))

  return drivers, riders



In [91]:
drivers, riders = generate_data()

for i in range(10) :
  print(drivers[i])

print("\n\n***********************************************************************\n\n")

for i in range(10) :
  print(riders[i])


N_drivers:  40
N_riders:  90
Driver ( id = 0, rating = 4.0, hour = 8:30, start = {'id': 2117, 'coordinates': (12.71, 45.59)}, dest = {'id': 137, 'coordinates': (12.03, 45.29)})
Driver ( id = 1, rating = 4.0, hour = 12:00, start = {'id': 2116, 'coordinates': (12.67, 45.62)}, dest = {'id': 270, 'coordinates': (11.8, 45.35)})
Driver ( id = 2, rating = 4.5, hour = 19:00, start = {'id': 328, 'coordinates': (12.18, 45.51)}, dest = {'id': 479, 'coordinates': (12.39, 45.53)})
Driver ( id = 3, rating = 4.0, hour = 7:30, start = {'id': 1835, 'coordinates': (12.02, 45.27)}, dest = {'id': 479, 'coordinates': (12.39, 45.53)})
Driver ( id = 4, rating = 3.5, hour = 18:00, start = {'id': 862, 'coordinates': (12.5, 45.61)}, dest = {'id': 2186, 'coordinates': (12.21, 45.53)})
Driver ( id = 5, rating = 3.0, hour = 7:30, start = {'id': 336, 'coordinates': (11.99, 45.46)}, dest = {'id': 479, 'coordinates': (12.39, 45.53)})
Driver ( id = 6, rating = 4.0, hour = 13:00, start = {'id': 218, 'coordinates': (12.

In [92]:
# use of Dijkstra for shortest path

drivers_path = []
riders_path = []

# 50 km/h
avg_vel = 50
date_format_str = '%H:%M'

for driver in drivers:
  spath = nx.dijkstra_path(G, driver.start_point.get('id'), driver.end_point.get('id'))
  dpath = [{'node' : spath[0], 'hour': driver.hour}]
  dist = 0


  for i in range(1, len(spath)):
    # retrieve distance and convert into km
    
    # distance in km
    dist += (G.get_edge_data(spath[i-1], spath[i]).get('weight')) / 1e3
    time = dist / avg_vel
    time_1 = datetime.strptime(driver.hour, date_format_str)
    time_2 = time_1 + timedelta(hours=time)
    str_time = time_2.strftime(date_format_str)

    dpath.append({'node': spath[i], 'hour':str_time})


  total_dist = nx.shortest_path_length(G, driver.start_point.get('id'), driver.end_point.get('id'), weight='weight')

  drivers_path.append({'id_driver': driver.id, 'shortest_path' : dpath, 'arrival_time': str_time})
  
  #print("Driver #" + str(driver.id) + ": start at " + driver.hour + " , end at " + dpath[len(dpath)-1].get('hour'))
  #print("    -> total dist km: ", total_dist/1e3)


for rider in riders:
  spath = nx.dijkstra_path(G, rider.start_point.get('id'), rider.end_point.get('id'))
  riders_path.append({'id_rider': rider.id, 'shortest_path' : spath})

print(drivers_path[0])
#print(riders_path)

{'id_driver': 0, 'shortest_path': [{'node': 2117, 'hour': '8:30'}, {'node': 1257, 'hour': '08:30'}, {'node': 420, 'hour': '08:30'}, {'node': 1417, 'hour': '08:30'}, {'node': 2107, 'hour': '08:36'}, {'node': 593, 'hour': '08:36'}, {'node': 595, 'hour': '08:44'}, {'node': 1575, 'hour': '08:46'}, {'node': 1555, 'hour': '08:51'}, {'node': 1553, 'hour': '08:57'}, {'node': 1284, 'hour': '09:02'}, {'node': 1557, 'hour': '09:02'}, {'node': 1225, 'hour': '09:02'}, {'node': 947, 'hour': '09:02'}, {'node': 1226, 'hour': '09:02'}, {'node': 1229, 'hour': '09:04'}, {'node': 1433, 'hour': '09:04'}, {'node': 1019, 'hour': '09:06'}, {'node': 769, 'hour': '09:06'}, {'node': 9, 'hour': '09:06'}, {'node': 10, 'hour': '09:09'}, {'node': 1207, 'hour': '09:09'}, {'node': 1929, 'hour': '09:15'}, {'node': 1930, 'hour': '09:15'}, {'node': 174, 'hour': '09:15'}, {'node': 171, 'hour': '09:26'}, {'node': 170, 'hour': '09:32'}, {'node': 173, 'hour': '09:35'}, {'node': 164, 'hour': '09:41'}, {'node': 162, 'hour': '0

In [93]:
# find matching: since we assumed drivers and riders having same dest

drivers_match = [{} for _ in range(len(drivers))]

# couple drivers and riders with same dest and similar hours

for driver in drivers :
  drivers_match[driver.id] = {'id_driver' : driver.id, 'list_of_riders' : []}
  for rider in riders :
    dhour_start = datetime.strptime(driver.hour, date_format_str)
    #print(drivers_path[driver.id].get('arrival_time'))
    dhour_end = datetime.strptime(drivers_path[driver.id].get('arrival_time'), date_format_str)
    rhour = datetime.strptime(rider.hour, date_format_str)

    
    if driver.end_point.get('id') == rider.end_point.get('id') and (dhour_start <= rhour <= dhour_end) :
      drivers_match[driver.id].get('list_of_riders').append(rider.id)

    

print(drivers_match)


[{'id_driver': 0, 'list_of_riders': [1, 37]}, {'id_driver': 1, 'list_of_riders': [59, 76, 86]}, {'id_driver': 2, 'list_of_riders': [62, 67]}, {'id_driver': 3, 'list_of_riders': [27, 71]}, {'id_driver': 4, 'list_of_riders': [75, 89]}, {'id_driver': 5, 'list_of_riders': [27, 71]}, {'id_driver': 6, 'list_of_riders': []}, {'id_driver': 7, 'list_of_riders': [27, 71]}, {'id_driver': 8, 'list_of_riders': [75, 89]}, {'id_driver': 9, 'list_of_riders': [1, 37]}, {'id_driver': 10, 'list_of_riders': [61, 72, 82]}, {'id_driver': 11, 'list_of_riders': [44]}, {'id_driver': 12, 'list_of_riders': [11, 17, 19, 31, 43, 85]}, {'id_driver': 13, 'list_of_riders': [5, 62, 67]}, {'id_driver': 14, 'list_of_riders': [14, 48, 49, 63, 73]}, {'id_driver': 15, 'list_of_riders': [76, 86]}, {'id_driver': 16, 'list_of_riders': [5, 62, 67]}, {'id_driver': 17, 'list_of_riders': [44]}, {'id_driver': 18, 'list_of_riders': [44]}, {'id_driver': 19, 'list_of_riders': [50, 87, 88]}, {'id_driver': 20, 'list_of_riders': []}, {'

In [ ]:
# create file input.dat (obsolete (?))
f = open("input.dat", "w")
content = "DIMENSION : " + str(len(nodes)) + "\n"
content += "DRIVERS : " + str(len(drivers)) + "\n"
content += "RIDERS : " + str(len(riders)) + "\n"
content += "NODE_COORD_SECTION\n"
for node in nodes :
  content += str(node.get("id")) + " " + str(node.get("coordinates")[0]) + " " + str(node.get("coordinates")[1]) + "\n"
content += "ROADS_SECTION\n"
for road in roads :
  content += str(road.get("p1")) + " " + str(road.get("p2")) + "\n"
content += "DRIVERS_STARTING_COORD\n"
for driver in drivers:
  content += str(driver.start_point.get("id")) + " " + str(driver.start_point.get("coordinates")[0]) + " " + str(driver.start_point.get("coordinates")[1]) + "\n"
content += "DRIVERS_DESTINATION_COORD\n"
for driver in drivers:
  content += str(driver.end_point.get("id")) + " " + str(driver.end_point.get("coordinates")[0]) + " " + str(driver.end_point.get("coordinates")[1]) + "\n"
content += "RIDERS_STARTING_COORD\n"
for rider in riders:
  content += str(rider.start_point.get("id")) + " " + str(rider.start_point.get("coordinates")[0]) + " " + str(rider.start_point.get("coordinates")[1]) + "\n"
content += "RIDERS_DESTINATION_COORD\n"
for rider in riders:
  content += str(rider.end_point.get("id")) + " " + str(rider.end_point.get("coordinates")[0]) + " " + str(rider.end_point.get("coordinates")[1]) + "\n"
content += "EOF"
f.write(content)
f.close()